In [1]:
import os
import random
import sys

import numpy as np
import pandas as pd
from deepeval import evaluate
from deepeval.dataset import EvaluationDataset
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm


sys.path.append(os.path.abspath("../"))
from src.chatbot import RecipeChatbot
from src.deepeval_openrouter import OpenRouterLLM
from src.config import DEFAULT_QA_MODEL, OPENAI_API_KEY
from src.data import indexing as data_indexing
from src.templates.prompt_templates import QA_GENERATION_TEMPLATE, STANDALONE_CRITIQUE_PROMPT_TEMPLATE, GROUNDNESS_CRITIQUE_PROMPT_TEMPLATE, RELEVANCE_CRITIQUE_PROMPT_TEMPLATE
from src.utils.embeddings import get_all_chroma_embeddings, get_embedding_model
from src.utils.generation import call_llm_openrouter
from src.utils.text_processing import format_dictionary_pairs

/Users/anna-alexandradanchenko/local/recipe-chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For the evaluation task, mainly deepeval library will be used. 

In [2]:
N_GENERATIONS=100
SIMILARITY_THRESHOLD = 0.925
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["DEEPEVAL_RESULTS_FOLDER"] = "../data/results"

In [3]:
outputs = []
used_questions = {}
embedding_model = get_embedding_model()

Loading embedding model: intfloat/multilingual-e5-large-instruct
Embedding model ready to use.


I believe that the best possible case would be manual creation of the answer-question pairs. However, due to the time constraints it is not possible, therefore we will use an LLM, as described in https://huggingface.co/learn/cookbook/en/rag_evaluation

I am aware of deepevals synthesiser, however, I used custom solution for better flexibility and research on how the QA generation works. 

In [4]:
chroma_data = get_all_chroma_embeddings(embedding_model)
doc_embeddings = chroma_data['embeddings']
docs = chroma_data['documents']
doc_ids = chroma_data['ids']
doc_metadatas = chroma_data['metadatas']

similarity_matrix = cosine_similarity(doc_embeddings)

doc_groups = []
used_docs = set()

# cluster docs
for i in range(len(docs)):
    if i in used_docs: 
        continue
        
    current_group = [i]
    used_docs.add(i)
    
    similar_indices = np.where(similarity_matrix[i] > SIMILARITY_THRESHOLD)[0]
    for idx in similar_indices:
        if idx != i and idx not in used_docs:
            current_group.append(int(idx))
            used_docs.add(idx)
    
    doc_groups.append(current_group)

print(f"Doc groups: {doc_groups}")

/Users/anna-alexandradanchenko/local/recipe-chatbot/src/utils/embeddings.py:25: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Doc groups: [[0, 6, 20], [1], [2, 30, 79], [3, 4, 10, 26], [5], [7], [8, 35, 55], [9], [11, 16], [12, 14], [13, 22, 71], [15], [17, 37], [18], [19], [21], [23], [24, 44], [25, 31, 68], [27], [28], [29], [32], [33, 97], [34], [36], [38], [39], [40], [41, 60], [42], [43, 52, 56], [45], [46, 62], [47, 54], [48, 61], [49], [50], [51], [53], [57], [58], [59], [63], [64], [65, 83, 85, 89], [66], [67], [69, 80], [70], [72], [73], [74, 76], [75], [77], [78], [81], [82], [84], [86], [87, 88, 96], [90], [91], [92], [93], [94], [95], [98], [99]]


## Generate QA Pairs

In [5]:
csv_path = f"../data/evaluation/qa_pairs.csv"

N_GENERATIONS = min(N_GENERATIONS, len(doc_groups))

if not os.path.exists(csv_path):
    for doc_group in tqdm(random.sample(doc_groups, N_GENERATIONS)):
        for _ in range(2): 
            n_docs = random.randint(1, len(doc_group))
            
            selected_indices = random.sample(doc_group, n_docs)
            selected_docs = [docs[idx] for idx in selected_indices]
            combined_content = "\n\n".join([doc for doc in selected_docs])

            formatted_qa = format_dictionary_pairs(used_questions)
            QA_PROMTP = QA_GENERATION_TEMPLATE.format(context=combined_content, qa_pairs=formatted_qa)
            output_QA_couple = call_llm_openrouter(QA_PROMTP, DEFAULT_QA_MODEL, {"question": "question", "answer": "answer"})
            used_questions[output_QA_couple["question"]] = output_QA_couple["answer"]
            outputs.append({
                "query": output_QA_couple["question"],
                "expected_output": output_QA_couple["answer"],
                "actual_output": "",  
                "context": combined_content,  
                "retrieval_context": ""
            })
            
            print(f"Question: {output_QA_couple['question']}")
            print(f"Answer: {output_QA_couple['answer']}")

    qa_df = pd.DataFrame(outputs)
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    qa_df.to_csv(csv_path, index=False)
    print(f"Saved {len(outputs)} QA pairs to {csv_path}")

  0%|          | 0/69 [00:00<?, ?it/s]

Question: Jak se připravuje ochucená strouhanka?
Answer: Pro přípravu ochucené strouhanky promíchejte strouhanku s citronovou kůrou, špetkou soli a pepře a najemno nakrájenou petrželí.


  1%|▏         | 1/69 [00:02<02:44,  2.42s/it]

Question: Jak se připravuje ochucená strouhanka, kterou se posypou makarony se sýrem?
Answer: Ochucenou strouhanku připravíte smícháním strouhanky s citronovou kůrou, špetkou soli, pepřem a najemno nasekanou petrželí.
Question: Můžu použít jiné než červené rajčata?
Answer: Ano, místo klasických červených rajčat můžete použít i zelenou nebo žlutou odrůdu, případně je zkombinovat.


  3%|▎         | 2/69 [00:04<02:24,  2.15s/it]

...

Question: Jakou texturu bude mít guacamole, když namačkám veškeré avokádo vidličkou?
Answer: Guacamole má mít jemnější texturu, pokud namačkáte veškeré avokádo vidličkou.
Question: Jak dlouho se grilují špízy souvlaki z jedné strany?
Answer: Špízy souvlaki se mají grilovat z každé strany 1,5 minuty, dokud není maso měkké.


 99%|█████████▊| 68/69 [02:20<00:01,  1.99s/it]

Question: Čím se potírají kuřecí špízy před grilováním?
Answer: Kuřecí špízy se potírají směsí olivového oleje, prolisovaného česneku, špetky soli, rozmarýnu a citronové šťávy.
Question: S čím se kromě citronu podávají vepřové řízky s bramborovým salátem?
Answer: Kromě citronu se vepřové řízky s bramborovým salátem podávají s bramborovým salátem.


100%|██████████| 69/69 [02:22<00:00,  2.07s/it]

Question: V čem se obaluje vepřová krkovice na vepřový řízek s bramborovým salátem?
Answer: Vepřová krkovice se obaluje v mouce, rozšlehaném vejci a strouhance.
Saved 138 QA pairs to ../data/evaluation/qa_pairs.csv


### Critique Agents to Filter out Bad QAs

In [ ]:
print("Generating critique for each QA couple...")
csv_path = f"../data/evaluation/qa_pairs.csv"

outputs = pd.read_csv(csv_path).to_dict("records")

for output in outputs:
    output["groundedness_rating"] = 0
    output["relevance_rating"] = 0
    output["standalone_rating"] = 0
    output["groundedness_eval"] = ""
    output["relevance_eval"] = ""
    output["standalone_eval"] = ""

for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm_openrouter(
            GROUNDNESS_CRITIQUE_PROMPT_TEMPLATE.format(question=output["query"], context=output["context"]),
            schema={"evaluation": "Provide a brief explanation of why you assigned this rating.", "total_rating": "Provide a number from 1 to 5."}        
            ),
        "relevance": call_llm_openrouter(
            
            RELEVANCE_CRITIQUE_PROMPT_TEMPLATE.format(question=output["query"]),
            schema={"evaluation": "Provide a brief explanation of why you assigned this rating.", "total_rating": "Provide a number from 1 to 5."}
        ),
        "standalone": call_llm_openrouter(
            STANDALONE_CRITIQUE_PROMPT_TEMPLATE.format(question=output["query"]),
            schema={"evaluation": "Provide a brief explanation of why you assigned this rating.", "total_rating": "Provide a number from 1 to 5."}
        ),
    }
    
    output["groundedness_rating"] = int(evaluations["groundedness"]["total_rating"])
    output["relevance_rating"] = int(evaluations["relevance"]["total_rating"])
    output["standalone_rating"] = int(evaluations["standalone"]["total_rating"])
    output["groundedness_eval"] = evaluations["groundedness"]["evaluation"]
    output["relevance_eval"] = evaluations["relevance"]["evaluation"]
    output["standalone_eval"] = evaluations["standalone"]["evaluation"]
    
non_filtered_qa_df = pd.DataFrame(outputs)
non_filtered_csv_path = "../data/evaluation/qa_pairs_eval_non_filtered.csv"
non_filtered_qa_df.to_csv(non_filtered_csv_path, index=False)

filtered_outputs = [
output for output in outputs 
if (output["groundedness_rating"] >= 4 and 
    output["relevance_rating"] >= 4 and 
    output["standalone_rating"] >= 4)
]

print(f"Filtered from {len(outputs)} to {len(filtered_outputs)} high-quality QA pairs")

filtered_qa_df = pd.DataFrame(filtered_outputs)
filtered_csv_path = "../data/evaluation/qa_pairs_filtered.csv"
filtered_qa_df.to_csv(filtered_csv_path, index=False)


Generating critique for each QA couple...


  0%|          | 0/138 [00:00<?, ?it/s]

100%|██████████| 138/138 [07:27<00:00,  3.25s/it]

Filtered from 138 to 123 high-quality QA pairs


## Generate Actual Output with RAG system

Let's generate actual outputs using our RAG system for the questions: 

In [13]:
chatbot = RecipeChatbot(embedding_model)
qa_df = pd.read_csv(filtered_csv_path)

for idx, row in tqdm(qa_df.iterrows(), total=len(qa_df)):
    query = row["query"]
    actual_output = chatbot.chat(query, False)
    print(actual_output)
    qa_df.loc[idx, "actual_output"] = actual_output["answer"]
    qa_df.loc[idx, "retrieval_context"] = actual_output["context"]

qa_df.to_csv(filtered_csv_path, index=False)

Loaded vector store from /Users/anna-alexandradanchenko/local/recipe-chatbot/data/chroma_db


  0%|          | 0/123 [00:00<?, ?it/s]/var/folders/6p/1jmnvrkx7jxdslfr99shm1z00000gn/T/ipykernel_36931/4270214919.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Omlouvám se, ale v momentálně dostupné databázi receptů nemám přesný recept na ochucenou strouhanku. Tato informace není z databáze receptů, ale mohu nabídnout obecnou radu:

Ochucená strouhanka je skvělé, když chcete dodat pokrmu extra chuť. Můžete ji připravit například z bílého chleba nebo rohlíků, které necháte ztvrdnout a poté je rozemelíte na jemné drobky. K základním ingrediencím můžete přidat různé koření a bylinky podle chuti, jako je sušená bazalka, oregano, tymián, sušený česnek nebo parmezán. Směs poté promíchejte, aby se všechny komponenty dobře propojily.

Pokud máte zájem o další tipy nebo podobné recepty, dejte mi vědět!' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  qa_df.loc[idx

{'answer': 'Omlouvám se, ale v momentálně dostupné databázi receptů nemám přesný recept na ochucenou strouhanku. Tato informace není z databáze receptů, ale mohu nabídnout obecnou radu:\n\nOchucená strouhanka je skvělé, když chcete dodat pokrmu extra chuť. Můžete ji připravit například z bílého chleba nebo rohlíků, které necháte ztvrdnout a poté je rozemelíte na jemné drobky. K základním ingrediencím můžete přidat různé koření a bylinky podle chuti, jako je sušená bazalka, oregano, tymián, sušený česnek nebo parmezán. Směs poté promíchejte, aby se všechny komponenty dobře propojily.\n\nPokud máte zájem o další tipy nebo podobné recepty, dejte mi vědět!', 'context': "recipe_name: Mazanec\nauthor_name: Roman Vaněk\ningredients: Čerstvé droždí, Citron, Cukr krupice, Hladká mouka, Káva vhodná k tomuto pokrmu, Mandle, Mandlové plátky, Máslo, Mléko, Papír na pečení, Polohrubá mouka, Rozinky, Sůl, Vanilkový cukr, Vejce\nsteps: ['Odpočinuté těsto skulíme do bochánku (tzn. pořádně jej utáhneme,

  2%|▏         | 2/123 [00:14<13:07,  6.51s/it]

{'answer': 'Ochucená strouhanka pro makarony se sýrem se připravuje tak, že strouhanku promícháte s citronovou kůrou, špetkou soli a pepře a najemno nakrájenou petrželí. Dobrou chuť!', 'context': "recipe_name: Makarony se sýrem\nauthor_name: Roman Vaněk\ningredients: Česnek, Cheddar bloček, Cheddar Extra Mature, Citron, Hladká mouka, Makarony, Máslo, Muškátový oříšek mletý, Olivový olej, Pepř černý , Plnotučné mléko, Plocholistá petržel, Strouhanka, Sůl, Víno vhodné k těstovinám\nsteps: ['Dvě třetiny bešamelu smícháme v míse s uvařenými těstovinami, vlijeme do vymazané zapékací mísy, zalijeme zbylým bešamelem a vložíme do rozehřáté trouby. Necháme zapéct 8 minut.', 'Mísu vyndáme z trouby a teplotu snížíme na 200 °C. Těstoviny posypeme zbylým sýrem a ochucenou strouhankou, vrátíme do trouby a zapečeme 8 -10 minut dozlatova.', 'Strouhanku promícháme s citronovou kůrou, špetkou soli a pepře a najemno nakrájenou petrželí. Oba druhy sýrů nahrubo nastrouháme a promícháme.', 'Těstoviny uvařím

...

{'answer': 'Kuřecí špízy se před grilováním potírají směsí z olivového oleje, prolisovaného česneku, špetky soli, čerstvého rozmarýnu a citronové šťávy. Dobrou chuť!', 'context': "recipe_name: Grilovaná kuřecí prsa\nauthor_name: Roman Vaněk\ningredients: Česnek, Citron, Kuřecí prsa bez kůže, Olivový olej, Rozmarýn čerstvý, Špejle, Sůl\nsteps: ['Do olivového oleje přidáme prolisovaný česnek, špetku soli, rozmarýn a citronovou šťávu.', 'Maso podélně nakrájíme na 6 proužků. Osolíme je a napíchneme na špejle.', 'Podávejte s dipy, tzatziki, omáčkami dle chuti nebo zeleným salátem smíchaným s cherry rajčátky a zakápnutým olivovým olejem.', 'Směsí potřeme kuřecí špízy, položíme je na rozpálený gril a grilujeme z každé strany 2-3 minuty dozlatova.']"}


 99%|█████████▉| 122/123 [08:48<00:04,  4.51s/it]

{'answer': 'Vepřové řízky se kromě citronu podávají tradičně s bramborovým salátem. Tento salát obsahuje brambory, mrkev, celer, nakládané okurky, vejce, majonézu a hořčici, dochucený solí a pepřem. Je ideální nechat salát před podáváním uležet alespoň 4 hodiny, nejlépe přes noc. Dobrou chuť!', 'context': "recipe_name: Vepřový řízek s bramborovým salátem\nauthor_name: Roman Vaněk\ningredients: Brambory typ A, B, C, Brambory, varný typ A, Celer, Cibule, Citron, Hladká mouka, Kuchynský papír, Majonéza, Mrkev, Nakládané okurky, Papír na pečení, Pepř černý , Petržel, Plnotučná hořčice, Sádlo, Strouhanka, Sůl, Vejce, Vepřová krkovice v celku i plátky, Vhodné pivo\nsteps: ['Hotové řízky položíme na talíř pokrytý kuchyňským papírem. Necháme tak odsát přebytečný tuk a řízky si chvíli odpočinou, budou pak šťavnatější. Podáváme s měsíčkem citronu a bramborovým salátem.', 'Maso naklepeme, z obou stran opepříme čerstvě mletým pepřem a vydatně osolíme.', 'Maso postupně obalíme v mouce, rozšlehaném 

100%|██████████| 123/123 [08:52<00:00,  4.33s/it]

{'answer': 'Vepřová krkovice na vepřový řízek s bramborovým salátem se obaluje postupně v hladké mouce, rozšlehaném vejci a strouhance. Dobrou chuť! Pokud byste potřeboval další podrobnosti nebo celý recept, dejte mi vědět.', 'context': "recipe_name: Vepřový řízek s bramborovým salátem\nauthor_name: Roman Vaněk\ningredients: Brambory typ A, B, C, Brambory, varný typ A, Celer, Cibule, Citron, Hladká mouka, Kuchynský papír, Majonéza, Mrkev, Nakládané okurky, Papír na pečení, Pepř černý , Petržel, Plnotučná hořčice, Sádlo, Strouhanka, Sůl, Vejce, Vepřová krkovice v celku i plátky, Vhodné pivo\nsteps: ['Hotové řízky položíme na talíř pokrytý kuchyňským papírem. Necháme tak odsát přebytečný tuk a řízky si chvíli odpočinou, budou pak šťavnatější. Podáváme s měsíčkem citronu a bramborovým salátem.', 'Maso naklepeme, z obou stran opepříme čerstvě mletým pepřem a vydatně osolíme.', 'Maso postupně obalíme v mouce, rozšlehaném vejci a strouhance.', 'Na pánvi na vyšším ohni rozpálíme sádlo a stáhn

In [8]:
owvnwof

NameError: name 'owvnwof' is not defined

In [2]:
filtered_csv_path = "../data/evaluation/qa_pairs_filtered.csv"
dataset = EvaluationDataset()

dataset.add_test_cases_from_csv_file(
    file_path=filtered_csv_path,
    input_col_name="query",
    actual_output_col_name="actual_output",
    expected_output_col_name="expected_output",
    context_col_name="context",
    retrieval_context_col_name="retrieval_context"
    )

metric = AnswerRelevancyMetric(
    model=OpenRouterLLM()
)

print(dataset)

EvaluationDataset(test_cases=[LLMTestCase(input='Jak se připravuje ochucená strouhanka?', actual_output='Omlouvám se, ale v momentálně dostupné databázi receptů nemám přesný recept na ochucenou strouhanku. Tato informace není z databáze receptů, ale mohu nabídnout obecnou radu:\n\nOchucená strouhanka je skvělé, když chcete dodat pokrmu extra chuť. Můžete ji připravit například z bílého chleba nebo rohlíků, které necháte ztvrdnout a poté je rozemelíte na jemné drobky. K základním ingrediencím můžete přidat různé koření a bylinky podle chuti, jako je sušená bazalka, oregano, tymián, sušený česnek nebo parmezán. Směs poté promíchejte, aby se všechny komponenty dobře propojily.\n\nPokud máte zájem o další tipy nebo podobné recepty, dejte mi vědět!', expected_output='Pro přípravu ochucené strouhanky promíchejte strouhanku s citronovou kůrou, špetkou soli a pepře a najemno nakrájenou petrželí.', context=["recipe_name: Makarony se sýrem\nauthor_name: Roman Vaněk\ningredients: Česnek, Cheddar 

In [3]:
evaluation_res = evaluate(test_cases=dataset.test_cases[:115], metrics=[metric])    

✨ You're running DeepEval's latest Answer Relevancy Metric! (using google/gemini-2.0-flash-001, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 115 test case(s) in parallel: |██████████|100% (115/115) [Time Taken: 06:40,  3.48s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.9, threshold: 0.5, strict: False, evaluation model: google/gemini-2.0-flash-001, reason: The score is 0.90 because while the response likely provides instructions on preparing flavored breadcrumbs, it includes a statement directing to the source of the information which is irrelevant to the preparation process itself., error: None)

For test case:

  - input: Jak se připravuje ochucená strouhanka?
  - actual output: Omlouvám se, ale v momentálně dostupné databázi receptů nemám přesný recept na ochucenou strouhanku. Tato informace není z databáze receptů, ale mohu nabídnout obecnou radu:

Ochucená strouhanka je skvělé, když chcete dodat pokrmu extra chuť. Můžete ji připravit například z bílého chleba nebo rohlíků, které necháte ztvrdnout a poté je rozemelíte na jemné drobky. K základním ingrediencím můžete přidat různé koření a bylinky podle chuti, jako je sušená bazalka, oregano, tymián, sušený česnek nebo parmezán. Směs poté promíche

Sending a large test run to Confident, this might take a bit longer than usual...

✓ Tests finished 🎉! View results on 
]8;id=632943;https://app.confident-ai.com/project/cm83pwwxt35h84e3zu74osc92/evaluation/test-runs/cm84duhz33n3q4e3zhzl3u3in/test-cases\https://app.confident-ai.com/project/cm83pwwxt35h84e3zu74osc92/evaluation/test-runs/cm84duhz33n3q4e3zhzl3u3in/test-]8;;\
]8;id=632943;https://app.confident-ai.com/project/cm83pwwxt35h84e3zu74osc92/evaluation/test-runs/cm84duhz33n3q4e3zhzl3u3in/test-cases\cases]8;;\.

In [14]:
results_list = []
for test_case in evaluation_res.test_results:
    result_dict = {
        "query": test_case.input,
        "test_case_name": test_case.name,
        "success": test_case.success,
        "score": test_case.metrics_data[0].score,
        "reason": test_case.metrics_data[0].reason,
        "actual_output": test_case.actual_output,
        "expected_output": test_case.expected_output,
        
    }
    
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

os.makedirs("../data/results", exist_ok=True)

results_df.to_csv("../data/results/evaluation_results.csv", index=False)
print(f"Results saved to ../data/results/evaluation_results.csv with {len(results_df)} records")

Results saved to ../data/results/evaluation_results.csv with 115 records
